# Microblaze Python Interface Libraries

In addition to using the `pynqmb` libraries from C it is also possible to create Python wrappers for the libraries directly. PYNQ provides the `MicroblazeLibrary` class for this purpose.

The `MicroblazeLibrary` class takes a list of libraries as a construction parameter which should be the names of the header files desired without the `.h` file extension. All of the constants and functions will then become members of the instance of the class.

For this example we are going to interact with the Adafruit V2 Motor Shield that exposes a PCA9685 chip. We are going to want the `i2c` library for interacting with the device.

## Setting up the FPGA for the Arduino Microblaze

First, we load the Arduino Microblaze overlay into the FPGA and then the I2C driver.

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.lib import MicroblazeLibrary
import time
import math

base = BaseOverlay('base.bit')  # load overlay into the Programmable Logic FPGA portion
lib = MicroblazeLibrary(base.ARDUINO, ['i2c'])  # load the i2c driver using the ARDUINO MicroBlaze IO Processor

Let's double-check that we can access the I2C driver by asking for the number of I2C driver devices available in this MicroBlaze IOP (Input-Output Processor):

In [ ]:
lib.i2c_get_num_devices()

## Working with the Arduino Microblaze's I2C controller

If we have at least one I2C controller device from the above cell, then we are cleared to use I2C!

Next we need to open our I2C device using the `i2c_open` function. This will return us an `i2c` object we can use for interacting with the bus.

In [ ]:
i2c_master = lib.i2c_open_device(0)

Below are helper functions so that you can read and write to I2C node device memories.

In [ ]:
def read_i2c_dev_reg(i2c_master, device_addr, reg_addr, num_bytes):
    """
    Read an I2C device's memory.
    
    :param i2c_master: the I2C Master device (Arduino MicroBlaze)
    :param device_addr: the I2C Node Address (accessory device)
    :param reg_addr: the register address on the I2C node to read
    :param num_bytes: the number of bytes to read
    """
    i2c_buffer = bytearray(num_bytes + 1)
    i2c_buffer[0] = reg_addr
    i2c_master.write(device_addr, i2c_buffer, 1)  # write the address
    i2c_master.read(device_addr, i2c_buffer, num_bytes)  # read the bytes back
    return i2c_buffer[0:num_bytes]


def write_i2c_dev_reg(i2c_master, device_addr, reg_addr, new_value, num_bytes):
    """
    Write to an I2C device's memory.
    
    :param i2c_master: the I2C Master device (Arduino MicroBlaze)
    :param device_addr: the I2C Node Address (accessory device)
    :param reg_addr: the register address on the I2C node to read
    :param new_value: the new value to write to the register
    :param num_bytes: the number of bytes to write
    """
    i2c_buffer = bytearray(num_bytes + 1)
    i2c_buffer[0] = reg_addr
    for idx in range(num_bytes, 0, -1):
        i2c_buffer[idx] = new_value >> (8 * (idx - 1)) & 0xFF  # restrict to one byte
    i2c_master.write(device_addr, i2c_buffer, num_bytes + 1)  # write the address
    return i2c_buffer


def write_i2c_dev_reg_bytes(i2c_master, device_addr, reg_addr, new_bytes):
    """
    Write to an I2C device's memory.
    
    :param i2c_master: the I2C Master device (Arduino MicroBlaze)
    :param device_addr: the I2C Node Address (accessory device)
    :param reg_addr: the register address on the I2C node to read
    :param new_bytes: the new bytearray to write to the memory
    """
    num_bytes = len(new_bytes)
    i2c_buffer = bytearray(num_bytes + 1)
    i2c_buffer[0] = reg_addr
    for idx, byte in enumerate(new_bytes):
        i2c_buffer[idx+1] = byte
    i2c_master.write(device_addr, i2c_buffer, num_bytes + 1)  # write the address
    return i2c_buffer


## Using the Arduino Microblaze to interact with the PCA9685 Motor Driver

### Motor Driver Setup

Now we can use the helper functions to help us double-check various values inside the PCA9685 from the datasheet:

In [ ]:
I2C_ADDRESS_AFMS = 0x60

""" Check for default value of 0x04 in reg 0x01 """
result = read_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0x01, 1)
print(result.hex())

""" Enable register auto-increment """
result = write_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0x00, 0x20, 1)
print(result.hex())

""" Check for register auto-increment, value of 0x20 in reg 0x00 """
result = read_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0x00, 1)
print(result.hex())

Let's go ahead and set the PWM frequency to 1600 Hz.  This procedure is just copied from the Arduino library written by Adafruit.

In [ ]:
def amfs_set_pwm_freq(i2c_master, i2c_addr=0x60, pwm_freq=1600) -> bool:
    """
    Set the PWM output frequency coming out of the AFMS.
    
    :param i2c_master: the I2C Master device (Arduino MicroBlaze)
    :param i2c_addr: the Adafruit Motor Shield I2C address
    :param pwm_freq: the base square wave frequency of the PWM output
    """
    pwm_freq_fixed = pwm_freq * 0.9  # due to overshoot problem
    prescaleval = math.floor(25000000 / 4096 / pwm_freq - 1 + 0.5)
    oldmode = read_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0x00, 1)[0]
    newmode = (oldmode & 0x7F) | 0x10
    write_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0x00, newmode, 1)
    write_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0xFE, prescaleval, 1)
    write_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0x00, oldmode, 1)
    time.sleep(5e-3)
    write_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0x00, oldmode | 0xa1, 1)
    result = read_i2c_dev_reg(i2c_master, I2C_ADDRESS_AFMS, 0xFE, 1)
    return result == prescaleval

We can call the function defined above now:

In [ ]:
amfs_set_pwm_freq(i2c_master, I2C_ADDRESS_AFMS, 1600)

### Motor Output Setup

Now that the motor driver itself is set up, we can choose the outputs we want.  Looking back from the schematic and the Adafruit Arduino source code, we see that the motor outputs are tied to different pins of our motor driver.  Our motor driver is actually a combination of the PCA9685 PWM driver integrated circuit and four H-bridges that are controlled by PWM.  The H-bridges need three inputs for each motor: PWM (motor speed), in1 (motor direction 1), and in2 (motor direction 2).  If in1 is ON and in2 is OFF, the motor will run one direction.  If in1 is OFF and in2 is ON, the motor will run the opposite direction.  If both in1 and in2 are OFF or ON, the H-bridge will release the motor (no power applied).

Here is a table that shows this:

| Motor   | PWM | in2  | in1  |
|---------|-----|------|------|
| M1      |  8  |  9   |  10  |
| M2      |  13 |  12  |  11  |
| M3      |  2  |  3   |  4   |
| M4      |  7  |  6   |  5   |


Below is a function that will help you write values to the motor driver.

To turn a pin (in2 or in1) ON, give it an on_start=4096 and off_start=0.

To turn a pin (in2 or in1) OFF, give it an on_start=0 and off_start=0.

To set the speed pin (PWM), give it an on_start=0 and off_start=(a value in [1,4095]).

You should set the motor speed before you turn the motor on.

In [ ]:
def amfs_set_pin_starts(
    pin_set: int, 
    on_start: int, 
    off_start: int, 
    i2c_master, 
    i2c_addr=0x60) -> bool:
    """
    Set the pin output counts for when the ON (high) and OFF (low) periods of the PWM signals should start.

    :param pin_set: the pin to set the counts for
    :param on_start: the value of the on counts
    :param off_start: the value of the off counts
    :param i2c_master: the i2c controller object
    :param i2c_addr: the i2c address of the motor shield PCA9685
    :returns: True if command was successful
    """
    pca_pin_reg_base = 0x06
    pca_pin_reg_addr = pca_pin_reg_base + 4 * pin_set
    pca_pin_vals = bytearray(4)
    pca_pin_vals[0] = on_start & 0xFF
    pca_pin_vals[1] = (on_start >> 8) & 0xFF
    pca_pin_vals[2] = off_start & 0xFF
    pca_pin_vals[3] = (off_start >> 8) & 0xFF
    write_i2c_dev_reg_bytes(i2c_master, i2c_addr, pca_pin_reg_addr, pca_pin_vals)
    result = read_i2c_dev_reg(i2c_master, i2c_addr, pca_pin_reg_addr, 4)
    return result == pca_pin_vals

Let's give it a try in the cell below!  Set a motor to have some speed, then turn it on!